## Important: Run this code cell each time you start a new session!

In [ ]:
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install os
!pip install opencv-python
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [ ]:
!wget -Ncnp https://physionet.org/files/images/1.0.0/E1154S7I000.png
!wget -Ncnp https://physionet.org/files/images/1.0.0/E1154S7I024.png

In [ ]:
import shutil
import os
orig_file = os.path.join('E1154S7I000.png')
os.rename(orig_file, 'bw_mra_single.png')
orig_file = os.path.join('E1154S7I024.png')
os.rename(orig_file, 'bw_mra_multiple.png')

# Instructions

To get full credit for this assignment, we should be able to run your entire notbook from start to finish without any errors. You can check this yourself by selecting "Runtime" > "Run all" in the Google Colab menu.

# Exercise 1: Working with Basic Shapes and Colors

This exercise will require drawing shapes on a blank image created using the following function:

In [ ]:
def create_black_img():
    return np.zeros((300,300,3), np.uint8)

**(Part 1)** Write code that draws a red circle with the following properties:
* Center at position `x = 250, y = 100`
* Radius of `25`
* Outline thickness of `3`

In [ ]:
img = create_black_img()
cv2.circle(img, center=(250, 100), radius=25,
           color=(255, 0, 0), thickness=3)
plt.figure(figsize=(3, 3))
plt.imshow(img)
plt.show()

**(Part 2)** Write code that draws a filled magenta rectangle with the following properties:
* Top left corner at position `x = 50, y = 100`
* Height of `50`
* Width of `100`


In [ ]:
img = create_black_img()
top_left = (50, 100)
h = 50
w = 100
bottom_right = (top_left[0]+w, top_left[1]+h)
cv2.rectangle(img, pt1=top_left, pt2=bottom_right,
              color=(255, 0, 255), thickness=-1)
plt.figure(figsize=(3, 3))
plt.imshow(img)
plt.show()

**(Part 3)** Write code that draws a filled white ellipse with the following properties:
* Center at position `x = 100, y = 200`
* Major axis of `100`
* Minor axis of `25`
* No rotation

In [ ]:
img = create_black_img()
cv2.ellipse(img, center=(100, 200), axes=(100, 25), angle=0,
            startAngle=0, endAngle=360,
            color=(255, 255, 255), thickness=-1)
plt.figure(figsize=(3, 3))
plt.imshow(img)
plt.show()

# Exercise 2: Identifying a Region of Interest in an MRA Image

This exercise will revolve around a magnetic resonance angiography (MRA) image showing a coronal slice from an anteroposterior position within the torso. We have already taken the liberty of applying a binary threshold to reveal the prominent regions of the MRA image.

In [ ]:
img = cv2.imread('bw_mra_multiple.png')
img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
_, img_thresh = cv2.threshold(img_gray, 200, 255, cv2.THRESH_BINARY)

plt.figure(figsize=(6, 3))
plt.subplot(1, 2, 1), plt.title('Original Image')
plt.imshow(img)
plt.subplot(1, 2, 2), plt.title('Thresholded Image')
plt.imshow(img_thresh, cmap='gray')
plt.show()

Notice that the binary image still contains numerous regions: three prominent ones along the center and right side of the image, and many smaller ones along the left side and the periphery. Our goal is to measure the radius of the round blob in the upper half of the image.

**(Part 1)** Write code that collects a list of the contours corresponding to the three largest regions in the binary image. There are multiple ways of doing this, but here are a couple of suggestions:
1. Sort all of the contours by their size and then pick the largest three ones
2. Iterate through the contours and keep the ones whose size exceed a minimum threshold defined by you

To confirm that this is working properly, draw green outlines around these regions using the code provided.

In [ ]:
# Locate the contours
cnts, hierarchy = cv2.findContours(img_thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Only keep the contours that pass the size check
large_cnts = []
for cnt in cnts:
    cnt_area = cv2.contourArea(cnt)
    if 500 < cnt_area:
        large_cnts.append(cnt)

# Draw the outlines on the image
output_img = img.copy()
for cnt in large_cnts:
    cv2.drawContours(output_img, [cnt], -1, (0, 255, 0), 3)

# Show the result
plt.figure(figsize=(6, 3))
plt.imshow(output_img, cmap='gray')
plt.show()

**(Part 2)** Write code that identifies the most circular contour out of the three regions that were selected in the previous problem. Draw a green outline around it using the code provided.

*Hint:* Think about the contour properties we covered during the lecture. A circle is a special kind of ellipse where the major and minor axes are equal.

In [ ]:
# Iterate through all of the contours
circular_cnt = None
min_axis_ratio = 1e6
for cnt in large_cnts:
    # Calculate the "roundness" using the ratio of the major and minor axes
    cnt_ellipse = cv2.fitEllipse(cnt)
    cnt_axes = cnt_ellipse[1]
    cnt_axis_ratio = cnt_axes[0]/cnt_axes[1]
    # Save the contour if this one is more round than past ones
    if cnt_axis_ratio < min_axis_ratio:
        circular_cnt = cnt

# Draw the outlines on the image
output_img = img.copy()
cv2.drawContours(output_img, [circular_cnt], -1, (0, 255, 0), 3)

# Show the result
plt.figure(figsize=(6, 3))
plt.imshow(output_img, cmap='gray')
plt.show()

**(Part 3)** Write code that calculates the radius of the circular region that you have identified in pixels.

In [ ]:
circle_outline = cv2.minEnclosingCircle(circular_cnt)
circle_outline[1]